# 🎯 AI 챗봇 멘토링 - 2차시: 프롬프트 최적화

## 🎯 학습 목표
- Jinja2 기반 동적 프롬프트 템플릿 시스템 구축
- 페르소나 및 브랜드 톤앤매너 적용
- A/B 테스트를 통한 프롬프트 최적화
- 프롬프트 트러블슈팅 및 일관성 확보
- 답변 품질 측정 및 개선 방법

## 📋 사전 준비사항
1. 1차시 완료 및 환경 설정
2. Jinja2 템플릿 엔진 이해
3. OpenAI API 키 설정
4. 프롬프트 엔지니어링 기본 지식

## 1️⃣ 환경 설정 및 라이브러리 임포트

In [ ]:
import os
import json
import time
import logging
from typing import Dict, List, Any, Optional, Tuple
from datetime import datetime
from dataclasses import dataclass, asdict
import hashlib
import random
from jinja2 import Template, Environment, FileSystemLoader
from openai import OpenAI

# 로컬 모듈 (상위 디렉토리)
import sys
sys.path.append('..')
from config import get_config

# 설정 로드
config = get_config()

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 라이브러리 임포트 완료")
print(f"🔑 API 키 설정 상태: {'✅ 설정됨' if config.llm.openai_api_key else '❌ 미설정'}")
print(f"🤖 사용 모델: {config.llm.openai_model}")

## 2️⃣ 데이터 구조 정의

프롬프트 템플릿과 페르소나를 체계적으로 관리하기 위한 데이터 클래스를 정의합니다.

In [ ]:
@dataclass
class PromptTemplate:
    """프롬프트 템플릿 구조"""
    name: str
    category: str
    template: str
    variables: List[str]
    description: str
    author: str
    version: str = "1.0"
    created_at: datetime = None
    
    def __post_init__(self):
        if self.created_at is None:
            self.created_at = datetime.now()

@dataclass
class PersonaConfig:
    """페르소나 설정"""
    name: str
    role: str
    tone: str
    expertise: List[str]
    constraints: List[str]
    examples: List[Dict[str, str]]
    brand_guidelines: Optional[Dict[str, Any]] = None

@dataclass
class PromptTestResult:
    """프롬프트 테스트 결과"""
    template_name: str
    test_input: Dict[str, Any]
    generated_prompt: str
    response: str
    tokens_used: int
    processing_time: float
    quality_score: float
    timestamp: datetime

print("✅ 데이터 구조 정의 완료")
print("📋 정의된 클래스:")
print("  - PromptTemplate: 프롬프트 템플릿 관리")
print("  - PersonaConfig: 페르소나 설정")
print("  - PromptTestResult: 테스트 결과 저장")

## 3️⃣ Jinja2 프롬프트 템플릿 엔진 구현

동적으로 프롬프트를 생성할 수 있는 템플릿 엔진을 구현합니다.

In [ ]:
class PromptTemplateEngine:
    """Jinja2 기반 프롬프트 템플릿 엔진"""
    
    def __init__(self, template_dir: str = "templates"):
        self.template_dir = template_dir
        self.env = Environment(
            loader=FileSystemLoader(template_dir) if os.path.exists(template_dir) else None,
            trim_blocks=True,
            lstrip_blocks=True
        )
        self.templates: Dict[str, PromptTemplate] = {}
        self._load_default_templates()
        logger.info(f"프롬프트 엔진 초기화 완료 - 템플릿 수: {len(self.templates)}")
    
    def _load_default_templates(self):
        """기본 템플릿 로드"""
        # 일반 어시스턴트 템플릿
        general_template = PromptTemplate(
            name="general_assistant",
            category="기본",
            template="""당신은 {{persona.role}}입니다. {{persona.tone}} 톤으로 답변해주세요.

전문분야: {{persona.expertise|join(', ')}}

제약사항:
{% for constraint in persona.constraints %}
- {{constraint}}
{% endfor %}

사용자 질문: {{user_question}}

답변 형식: {{response_format}}
답변 길이: {{length_limit}}자 이내""",
            variables=["persona", "user_question", "response_format", "length_limit"],
            description="일반적인 어시스턴트 역할을 위한 기본 템플릿",
            author="AI Workshop"
        )
        
        # 코드 리뷰 템플릿
        code_review_template = PromptTemplate(
            name="code_reviewer",
            category="개발",
            template="""당신은 {{years_experience}}년 경력의 {{programming_language}} 전문가입니다.

다음 코드를 리뷰해주세요:

```{{programming_language}}
{{code_snippet}}
```

리뷰 관점:
{% for aspect in review_aspects %}
- {{aspect}}
{% endfor %}

출력 형식:
1. 코드 품질 점수 (1-10)
2. 주요 개선사항
3. 보안 이슈
4. 성능 최적화 제안
5. 개선된 코드 예시""",
            variables=["years_experience", "programming_language", "code_snippet", "review_aspects"],
            description="코드 리뷰를 위한 전문 템플릿",
            author="AI Workshop"
        )
        
        # 고객지원 템플릿
        customer_support_template = PromptTemplate(
            name="customer_support",
            category="고객지원",
            template="""안녕하세요! {{company_name}} 고객지원팀의 {{agent_name}}입니다.

브랜드 가이드라인:
- 톤: {{brand_tone}}
- 핵심가치: {{brand_values|join(', ')}}
- 금지어: {{forbidden_words|join(', ')}}

고객 문의:
분류: {{inquiry_category}}
내용: {{customer_message}}
우선순위: {{priority_level}}

응답 가이드라인:
1. 공감적 인사
2. 문제 파악 확인
3. 구체적 해결방안 제시
4. 추가 도움 제안

최대 {{max_length}}자로 답변해주세요.""",
            variables=["company_name", "agent_name", "brand_tone", "brand_values", 
                      "forbidden_words", "inquiry_category", "customer_message", 
                      "priority_level", "max_length"],
            description="고객지원을 위한 브랜드 가이드라인 적용 템플릿",
            author="AI Workshop"
        )
        
        self.templates["general_assistant"] = general_template
        self.templates["code_reviewer"] = code_review_template
        self.templates["customer_support"] = customer_support_template
    
    def add_template(self, template: PromptTemplate):
        """새 템플릿 추가"""
        self.templates[template.name] = template
        logger.info(f"템플릿 추가: {template.name}")
    
    def get_template(self, name: str) -> Optional[PromptTemplate]:
        """템플릿 조회"""
        return self.templates.get(name)
    
    def list_templates(self, category: Optional[str] = None) -> List[PromptTemplate]:
        """템플릿 목록 조회"""
        if category:
            return [t for t in self.templates.values() if t.category == category]
        return list(self.templates.values())
    
    def render_prompt(self, template_name: str, variables: Dict[str, Any]) -> str:
        """프롬프트 렌더링"""
        template_obj = self.get_template(template_name)
        if not template_obj:
            raise ValueError(f"템플릿을 찾을 수 없습니다: {template_name}")
        
        try:
            jinja_template = Template(template_obj.template)
            rendered = jinja_template.render(**variables)
            logger.debug(f"프롬프트 렌더링 완료 - 템플릿: {template_name}")
            return rendered
        except Exception as e:
            logger.error(f"프롬프트 렌더링 실패: {e}")
            raise

# 템플릿 엔진 생성 및 테스트
template_engine = PromptTemplateEngine()

print(f"✅ 템플릿 엔진 생성 완료")
print(f"📝 로드된 템플릿: {len(template_engine.templates)}개")

for template_name, template in template_engine.templates.items():
    print(f"  - {template_name} ({template.category}): {template.description}")

## 4️⃣ 페르소나 관리 시스템

다양한 페르소나를 정의하고 관리하는 시스템을 구축합니다.

In [ ]:
class PersonaManager:
    """페르소나 관리자"""
    
    def __init__(self):
        self.personas: Dict[str, PersonaConfig] = {}
        self._load_default_personas()
        logger.info(f"페르소나 관리자 초기화 - 페르소나 수: {len(self.personas)}")
    
    def _load_default_personas(self):
        """기본 페르소나 로드"""
        
        # 친근한 어시스턴트
        friendly_assistant = PersonaConfig(
            name="friendly_assistant",
            role="도움이 되는 AI 어시스턴트",
            tone="친근하고 따뜻한",
            expertise=["일반 상식", "문제 해결", "정보 제공"],
            constraints=[
                "항상 정중하고 예의바르게 대답",
                "확실하지 않은 정보는 추측하지 않음",
                "개인정보는 절대 요청하지 않음"
            ],
            examples=[
                {
                    "input": "안녕하세요",
                    "output": "안녕하세요! 무엇을 도와드릴까요? 궁금한 것이 있으시면 언제든 말씀해주세요."
                }
            ]
        )
        
        # 기술 전문가
        technical_expert = PersonaConfig(
            name="technical_expert",
            role="10년 경력의 소프트웨어 엔지니어",
            tone="전문적이면서도 이해하기 쉬운",
            expertise=["Python", "웹개발", "AI/ML", "시스템 아키텍처", "데이터베이스"],
            constraints=[
                "정확한 기술 정보만 제공",
                "예시 코드는 실행 가능해야 함",
                "최신 기술 동향 반영",
                "초보자도 이해할 수 있도록 설명"
            ],
            examples=[
                {
                    "input": "Python으로 API를 어떻게 만들어요?",
                    "output": "FastAPI나 Flask를 사용하면 쉽게 만들 수 있습니다. 예를 들어 FastAPI로 간단한 API를 만들어보겠습니다..."
                }
            ]
        )
        
        # 브랜드 앰배서더
        brand_ambassador = PersonaConfig(
            name="brand_ambassador",
            role="기업 브랜드 대변인",
            tone="전문적이고 신뢰할 수 있는",
            expertise=["브랜드 커뮤니케이션", "고객 서비스", "마케팅"],
            constraints=[
                "브랜드 가치와 일치하는 메시지",
                "경쟁사 언급 금지",
                "부정적 표현 최소화",
                "항상 솔루션 중심적 접근"
            ],
            examples=[
                {
                    "input": "제품에 문제가 있어요",
                    "output": "불편을 끼쳐드려 죄송합니다. 빠른 해결을 위해 구체적인 상황을 알려주시겠어요? 최선을 다해 도움드리겠습니다."
                }
            ],
            brand_guidelines={
                "tone": "professional_friendly",
                "values": ["고객중심", "혁신", "신뢰", "품질"],
                "forbidden_words": ["문제", "불가능", "안됨", "모름"],
                "preferred_words": ["해결", "가능", "도움", "지원"]
            }
        )
        
        self.personas["friendly_assistant"] = friendly_assistant
        self.personas["technical_expert"] = technical_expert
        self.personas["brand_ambassador"] = brand_ambassador
    
    def add_persona(self, persona: PersonaConfig):
        """새 페르소나 추가"""
        self.personas[persona.name] = persona
        logger.info(f"페르소나 추가: {persona.name}")
    
    def get_persona(self, name: str) -> Optional[PersonaConfig]:
        """페르소나 조회"""
        return self.personas.get(name)
    
    def list_personas(self) -> List[PersonaConfig]:
        """페르소나 목록"""
        return list(self.personas.values())

# 페르소나 매니저 생성 및 테스트
persona_manager = PersonaManager()

print(f"✅ 페르소나 관리자 생성 완료")
print(f"👥 로드된 페르소나: {len(persona_manager.personas)}개")

for persona_name, persona in persona_manager.personas.items():
    print(f"  - {persona.name}: {persona.role} ({persona.tone})")
    print(f"    전문분야: {', '.join(persona.expertise[:3])}{'...' if len(persona.expertise) > 3 else ''}")

## 5️⃣ 프롬프트 렌더링 테스트

실제로 템플릿과 페르소나를 결합하여 프롬프트를 생성해봅시다.

In [ ]:
# 일반 어시스턴트 템플릿 테스트
print("🧪 일반 어시스턴트 템플릿 테스트")
print("=" * 50)

# 변수 설정
persona = persona_manager.get_persona("friendly_assistant")
variables = {
    "persona": persona,
    "user_question": "파이썬으로 웹 크롤링을 어떻게 시작하면 좋을까요?",
    "response_format": "단계별 가이드",
    "length_limit": 300
}

# 프롬프트 렌더링
try:
    rendered_prompt = template_engine.render_prompt("general_assistant", variables)
    print("📝 생성된 프롬프트:")
    print("-" * 30)
    print(rendered_prompt)
    print("-" * 30)
    print(f"📊 프롬프트 길이: {len(rendered_prompt)} 문자")
    
except Exception as e:
    print(f"❌ 렌더링 실패: {e}")

print("\n" + "=" * 50)

In [ ]:
# 코드 리뷰 템플릿 테스트
print("🧪 코드 리뷰 템플릿 테스트")
print("=" * 50)

# 변수 설정
code_variables = {
    "years_experience": 10,
    "programming_language": "Python",
    "code_snippet": """def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

result = fibonacci(35)
print(result)""",
    "review_aspects": ["성능", "가독성", "메모리 사용", "알고리즘 효율성"]
}

# 프롬프트 렌더링
try:
    code_prompt = template_engine.render_prompt("code_reviewer", code_variables)
    print("📝 생성된 코드 리뷰 프롬프트:")
    print("-" * 30)
    print(code_prompt)
    print("-" * 30)
    print(f"📊 프롬프트 길이: {len(code_prompt)} 문자")
    
except Exception as e:
    print(f"❌ 렌더링 실패: {e}")

print("\n" + "=" * 50)

## 6️⃣ AI 응답 생성 및 비교

생성된 프롬프트로 실제 AI 응답을 생성하고 품질을 비교해봅시다.

In [ ]:
# OpenAI 클라이언트 초기화
if config.llm.openai_api_key:
    client = OpenAI(api_key=config.llm.openai_api_key)
    print("✅ OpenAI 클라이언트 초기화 완료")
else:
    print("❌ OpenAI API 키가 설정되지 않았습니다.")
    client = None

def generate_ai_response(prompt: str, user_input: str, model: str = None) -> Dict[str, Any]:
    """AI 응답 생성 헬퍼 함수"""
    if not client:
        return {"error": "OpenAI 클라이언트가 초기화되지 않았습니다."}
    
    try:
        start_time = time.time()
        
        response = client.chat.completions.create(
            model=model or config.llm.openai_model,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": user_input}
            ],
            max_tokens=500,
            temperature=0.7
        )
        
        processing_time = time.time() - start_time
        
        return {
            "response": response.choices[0].message.content,
            "tokens_used": response.usage.total_tokens,
            "processing_time": processing_time,
            "model": model or config.llm.openai_model
        }
        
    except Exception as e:
        return {"error": str(e)}

# 일반 어시스턴트 프롬프트로 응답 생성
if client:
    print("🤖 일반 어시스턴트 프롬프트 응답 생성")
    print("=" * 50)
    
    user_question = "파이썬으로 웹 크롤링을 어떻게 시작하면 좋을까요?"
    result = generate_ai_response(rendered_prompt, user_question)
    
    if "error" not in result:
        print(f"👤 사용자: {user_question}")
        print(f"\n🤖 AI 응답:")
        print(result["response"])
        print(f"\n📊 메타데이터:")
        print(f"  - 토큰 사용: {result['tokens_used']}")
        print(f"  - 처리 시간: {result['processing_time']:.2f}초")
        print(f"  - 모델: {result['model']}")
    else:
        print(f"❌ 응답 생성 실패: {result['error']}")
else:
    print("⚠️ API 키가 설정되지 않아 응답 생성을 건너뜁니다.")

print("\n" + "=" * 50)

## 7️⃣ 프롬프트 최적화 및 A/B 테스트 시스템

프롬프트의 품질을 측정하고 A/B 테스트를 통해 최적화하는 시스템을 구현합니다.

In [ ]:
class PromptOptimizer:
    """프롬프트 최적화 및 A/B 테스트"""
    
    def __init__(self, openai_client: OpenAI):
        self.client = openai_client
        self.test_results: List[PromptTestResult] = []
        logger.info("프롬프트 최적화 엔진 초기화 완료")
    
    def calculate_quality_score(self, input_text: str, output_text: str, prompt: str) -> float:
        """품질 점수 계산 (휴리스틱)"""
        score = 50  # 기본 점수
        
        # 1. 길이 적절성 (너무 짧거나 길지 않음)
        if 50 <= len(output_text) <= 500:
            score += 20
        elif len(output_text) < 20:
            score -= 10
        
        # 2. 관련성 (입력과 출력의 키워드 매칭)
        input_words = set(input_text.lower().split())
        output_words = set(output_text.lower().split())
        if input_words:
            relevance = len(input_words & output_words) / len(input_words)
            score += relevance * 15
        
        # 3. 구조화 정도 (문장 구조, 리스트 등)
        structure_indicators = ['. ', '\n', ':', '-', '1.', '2.', '•']
        structure_score = sum(1 for indicator in structure_indicators if indicator in output_text)
        score += min(structure_score * 2, 10)
        
        # 4. 전문성 (전문 용어 사용)
        technical_terms = ['알고리즘', '함수', '클래스', '모듈', '라이브러리', 'API', '데이터베이스']
        tech_score = sum(1 for term in technical_terms if term in output_text)
        score += min(tech_score * 1.5, 5)
        
        return min(100, max(0, score))
    
    def test_prompt_quality(self, prompt: str, test_inputs: List[str]) -> float:
        """프롬프트 품질 평가"""
        if not self.client:
            return 0
        
        scores = []
        
        for test_input in test_inputs:
            try:
                result = generate_ai_response(prompt, test_input)
                if "error" not in result:
                    score = self.calculate_quality_score(
                        test_input, result["response"], prompt
                    )
                    scores.append(score)
            except Exception as e:
                logger.error(f"품질 테스트 실패: {e}")
                scores.append(0)
        
        avg_score = sum(scores) / len(scores) if scores else 0
        logger.info(f"평균 품질 점수: {avg_score:.1f}")
        return avg_score
    
    def run_ab_test(self, prompt_a: str, prompt_b: str, 
                   test_inputs: List[str], test_name: str = "A/B Test") -> Dict[str, Any]:
        """A/B 테스트 실행"""
        if not self.client:
            return {"error": "OpenAI 클라이언트가 없습니다."}
        
        print(f"🔬 A/B 테스트 시작: {test_name}")
        start_time = time.time()
        
        results_a = []
        results_b = []
        
        for i, test_input in enumerate(test_inputs):
            print(f"  테스트 {i+1}/{len(test_inputs)} 진행중...")
            
            # 프롬프트 A 테스트
            result_a = generate_ai_response(prompt_a, test_input)
            if "error" not in result_a:
                quality_a = self.calculate_quality_score(
                    test_input, result_a["response"], prompt_a
                )
                result_a["quality_score"] = quality_a
                results_a.append(result_a)
            
            # 프롬프트 B 테스트
            result_b = generate_ai_response(prompt_b, test_input)
            if "error" not in result_b:
                quality_b = self.calculate_quality_score(
                    test_input, result_b["response"], prompt_b
                )
                result_b["quality_score"] = quality_b
                results_b.append(result_b)
        
        # 결과 분석
        if results_a and results_b:
            avg_score_a = sum(r["quality_score"] for r in results_a) / len(results_a)
            avg_score_b = sum(r["quality_score"] for r in results_b) / len(results_b)
            avg_time_a = sum(r["processing_time"] for r in results_a) / len(results_a)
            avg_time_b = sum(r["processing_time"] for r in results_b) / len(results_b)
            avg_tokens_a = sum(r["tokens_used"] for r in results_a) / len(results_a)
            avg_tokens_b = sum(r["tokens_used"] for r in results_b) / len(results_b)
        else:
            return {"error": "테스트 결과가 없습니다."}
        
        total_time = time.time() - start_time
        winner = "A" if avg_score_a > avg_score_b else "B"
        score_diff = abs(avg_score_a - avg_score_b)
        
        result = {
            "test_name": test_name,
            "winner": winner,
            "score_difference": score_diff,
            "results": {
                "prompt_a": {
                    "avg_quality_score": avg_score_a,
                    "avg_processing_time": avg_time_a,
                    "avg_tokens": avg_tokens_a,
                    "results_count": len(results_a)
                },
                "prompt_b": {
                    "avg_quality_score": avg_score_b,
                    "avg_processing_time": avg_time_b,
                    "avg_tokens": avg_tokens_b,
                    "results_count": len(results_b)
                }
            },
            "test_duration": total_time,
            "total_tests": len(test_inputs) * 2,
            "timestamp": datetime.now()
        }
        
        print(f"✅ A/B 테스트 완료 - 승자: {winner}, 점수차: {score_diff:.1f}")
        return result

# 최적화 엔진 생성
if client:
    optimizer = PromptOptimizer(client)
    print("✅ 프롬프트 최적화 엔진 생성 완료")
else:
    optimizer = None
    print("⚠️ API 키가 없어 최적화 엔진을 건너뜁니다.")

## 8️⃣ A/B 테스트 실행

두 가지 다른 프롬프트 스타일을 비교하는 A/B 테스트를 실행해봅시다.

In [ ]:
if optimizer:
    print("⚖️ A/B 테스트 실행")
    print("=" * 50)
    
    # 테스트할 두 프롬프트 정의
    prompt_a = """당신은 친근한 파이썬 튜터입니다. 초보자도 이해할 수 있도록 쉽게 설명해주세요.
단계별로 차근차근 설명하고, 실용적인 예시를 들어주세요.
질문에 대해 정확하고 도움이 되는 답변을 제공해주세요."""
    
    prompt_b = """당신은 10년 경력의 시니어 파이썬 개발자입니다. 
전문적이고 정확한 기술 정보를 제공하되, 실무에서 바로 적용할 수 있는 실용적인 조언을 해주세요.
코드 예시는 production-ready 수준으로 작성하고, 베스트 프랙티스를 포함해주세요.
성능과 유지보수를 고려한 솔루션을 제안해주세요."""
    
    # 테스트 입력들
    test_inputs = [
        "파이썬으로 웹 크롤링을 어떻게 시작하면 좋을까요?",
        "리스트와 딕셔너리의 차이점을 설명해주세요.",
        "파이썬에서 에러 처리는 어떻게 하나요?",
        "데이터베이스 연결하는 방법을 알려주세요."
    ]
    
    print(f"📝 프롬프트 A: 친근한 튜터 스타일")
    print(f"📝 프롬프트 B: 시니어 개발자 스타일")
    print(f"🧪 테스트 케이스: {len(test_inputs)}개")
    print()
    
    # A/B 테스트 실행
    ab_result = optimizer.run_ab_test(
        prompt_a, prompt_b, test_inputs[:2],  # 시간 절약을 위해 2개만 테스트
        "튜터 vs 시니어 개발자 스타일"
    )
    
    if "error" not in ab_result:
        print(f"\n📊 A/B 테스트 결과")
        print("=" * 30)
        print(f"🏆 승자: 프롬프트 {ab_result['winner']}")
        print(f"📈 점수 차이: {ab_result['score_difference']:.1f}점")
        print(f"⏱️ 총 소요 시간: {ab_result['test_duration']:.1f}초")
        
        # 세부 결과
        results_a = ab_result['results']['prompt_a']
        results_b = ab_result['results']['prompt_b']
        
        print(f"\n📊 프롬프트 A (친근한 튜터):")
        print(f"  - 평균 품질 점수: {results_a['avg_quality_score']:.1f}/100")
        print(f"  - 평균 처리 시간: {results_a['avg_processing_time']:.2f}초")
        print(f"  - 평균 토큰 수: {results_a['avg_tokens']:.0f}개")
        
        print(f"\n📊 프롬프트 B (시니어 개발자):")
        print(f"  - 평균 품질 점수: {results_b['avg_quality_score']:.1f}/100")
        print(f"  - 평균 처리 시간: {results_b['avg_processing_time']:.2f}초")
        print(f"  - 평균 토큰 수: {results_b['avg_tokens']:.0f}개")
        
        # 권장사항
        print(f"\n💡 권장사항:")
        if ab_result['winner'] == 'A':
            print("  친근한 튜터 스타일이 더 효과적입니다.")
            print("  초보자 대상 서비스에 적합합니다.")
        else:
            print("  시니어 개발자 스타일이 더 효과적입니다.")
            print("  전문가 대상 서비스에 적합합니다.")
    else:
        print(f"❌ A/B 테스트 실패: {ab_result['error']}")
else:
    print("⚠️ API 키가 없어 A/B 테스트를 건너뜁니다.")

print("\n" + "=" * 50)

## 9️⃣ 프롬프트 트러블슈팅 도구

실무에서 자주 발생하는 프롬프트 문제들을 해결하는 도구를 구현합니다.

In [ ]:
class TroubleshootingSolver:
    """프롬프트 트러블슈팅 해결사"""
    
    @staticmethod
    def ensure_consistency(base_prompt: str) -> str:
        """일관성 확보를 위한 프롬프트 개선"""
        consistency_suffix = """

[중요한 지침]
- temperature=0으로 설정하여 일관된 응답을 생성하세요
- 동일한 질문에는 항상 동일한 구조로 답변하세요
- 예측 가능하고 신뢰할 수 있는 응답을 제공하세요
- 답변 형식과 길이를 일정하게 유지하세요"""
        
        return base_prompt + consistency_suffix
    
    @staticmethod
    def compress_prompt(long_prompt: str, max_length: int = 1000) -> str:
        """프롬프트 압축 (토큰 수 최적화)"""
        if len(long_prompt) <= max_length:
            return long_prompt
        
        import re
        compressed = long_prompt
        
        # 1. 중복 공백 제거
        compressed = re.sub(r'\s+', ' ', compressed)
        
        # 2. 불필요한 구문 제거
        unnecessary_phrases = [
            "please", "kindly", "I would like you to",
            "Could you", "Would you mind", "It would be great if",
            "부탁드립니다", "해주시면 감사하겠습니다"
        ]
        
        for phrase in unnecessary_phrases:
            compressed = compressed.replace(phrase, "")
        
        # 3. 핵심 내용만 유지 (문장 단위로 자르기)
        if len(compressed) > max_length:
            sentences = compressed.split('. ')
            truncated = []
            current_length = 0
            
            for sentence in sentences:
                if current_length + len(sentence) <= max_length:
                    truncated.append(sentence)
                    current_length += len(sentence) + 2  # '. ' 포함
                else:
                    break
            
            compressed = '. '.join(truncated)
        
        logger.info(f"프롬프트 압축: {len(long_prompt)} -> {len(compressed)} 문자")
        return compressed.strip()
    
    @staticmethod
    def fix_response_format(prompt: str, desired_format: str) -> str:
        """응답 형식 제어 개선"""
        format_instructions = {
            "json": "\n\n응답을 반드시 유효한 JSON 형식으로만 제공하세요. 다른 설명은 포함하지 마세요.",
            "markdown": "\n\n응답을 마크다운 형식으로 작성하세요. 헤더, 리스트, 코드 블록을 활용하세요.",
            "bullet_points": "\n\n응답을 불릿 포인트 형태로 정리해주세요:\n- 첫 번째 요점\n- 두 번째 요점\n- ...",
            "numbered_list": "\n\n응답을 번호 매긴 목록으로 작성해주세요:\n1. 첫 번째 항목\n2. 두 번째 항목\n...",
            "paragraph": "\n\n응답을 잘 구조화된 문단 형태로 작성해주세요. 각 문단은 하나의 주요 아이디어를 다루세요.",
            "code_only": "\n\n코드만 제공하고 설명은 주석으로만 작성하세요. 마크다운 코드 블록을 사용하세요."
        }
        
        format_instruction = format_instructions.get(desired_format, "")
        return prompt + format_instruction
    
    @staticmethod
    def add_constraints(prompt: str, constraints: List[str]) -> str:
        """제약조건 추가"""
        if not constraints:
            return prompt
        
        constraint_section = "\n\n[중요한 제약사항]\n"
        for i, constraint in enumerate(constraints, 1):
            constraint_section += f"{i}. {constraint}\n"
        
        return prompt + constraint_section
    
    @staticmethod
    def add_examples(prompt: str, examples: List[Dict[str, str]]) -> str:
        """예시 추가 (Few-shot learning)"""
        if not examples:
            return prompt
        
        examples_section = "\n\n[예시]\n"
        for i, example in enumerate(examples, 1):
            examples_section += f"예시 {i}:\n"
            examples_section += f"입력: {example.get('input', '')}\n"
            examples_section += f"출력: {example.get('output', '')}\n\n"
        
        return prompt + examples_section

# 트러블슈팅 도구 테스트
print("🔧 프롬프트 트러블슈팅 도구 테스트")
print("=" * 50)

# 원본 프롬프트 (일부러 길고 장황하게 작성)
original_prompt = """
안녕하세요! 당신은 정말 도움이 많이 되는 파이썬 프로그래밍 전문가입니다. 
사용자의 질문에 대해 친절하고 자세하게 답변해주시면 정말 감사하겠습니다. 
Could you please provide detailed explanations that would be helpful for beginners? 
답변을 작성하실 때는 단계별로 설명해주시고, 가능하다면 코드 예시도 포함해주시면 좋겠습니다. 
It would be great if you could also explain why certain approaches are better than others. 
마지막으로, 답변이 너무 길어지지 않도록 주의해주시면 감사하겠습니다.
"""

print("📝 원본 프롬프트:")
print(f"길이: {len(original_prompt)} 문자")
print(original_prompt)
print()

# 1. 압축 테스트
compressed_prompt = TroubleshootingSolver.compress_prompt(original_prompt, 200)
print("🗜️ 압축된 프롬프트:")
print(f"길이: {len(compressed_prompt)} 문자 ({len(original_prompt) - len(compressed_prompt)} 문자 절약)")
print(compressed_prompt)
print()

# 2. 일관성 개선
consistent_prompt = TroubleshootingSolver.ensure_consistency(compressed_prompt)
print("🎯 일관성 개선된 프롬프트:")
print(f"길이: {len(consistent_prompt)} 문자")
print(consistent_prompt)
print()

# 3. 응답 형식 지정
formatted_prompt = TroubleshootingSolver.fix_response_format(compressed_prompt, "numbered_list")
print("📋 형식 지정된 프롬프트:")
print(formatted_prompt)
print()

# 4. 제약조건 추가
constraints = [
    "답변은 300자 이내로 작성",
    "코드 예시 포함 필수",
    "초보자가 이해할 수 있는 용어 사용"
]
constrained_prompt = TroubleshootingSolver.add_constraints(compressed_prompt, constraints)
print("⚠️ 제약조건 추가된 프롬프트:")
print(constrained_prompt)
print()

print("=" * 50)

## 🔟 개선된 프롬프트 성능 비교

트러블슈팅 도구로 개선한 프롬프트의 성능을 원본과 비교해봅시다.

In [ ]:
if optimizer and client:
    print("📊 개선된 프롬프트 성능 비교")
    print("=" * 50)
    
    # 테스트용 간단한 프롬프트들
    original_simple = "파이썬에 대해 친절하게 설명해주세요."
    improved_simple = TroubleshootingSolver.fix_response_format(
        TroubleshootingSolver.ensure_consistency(
            "당신은 파이썬 전문가입니다. 정확하고 체계적으로 설명하세요."
        ),
        "numbered_list"
    )
    
    # 테스트 질문
    test_questions = [
        "파이썬 리스트와 튜플의 차이점은 무엇인가요?",
        "파이썬에서 함수를 정의하는 방법을 알려주세요."
    ]
    
    print("🔍 비교 대상:")
    print(f"  원본: {original_simple}")
    print(f"  개선: (일관성 + 형식 지정)")
    print()
    
    # 품질 점수 비교
    original_score = optimizer.test_prompt_quality(original_simple, test_questions)
    improved_score = optimizer.test_prompt_quality(improved_simple, test_questions)
    
    print(f"📈 품질 점수 비교:")
    print(f"  원본 프롬프트: {original_score:.1f}/100")
    print(f"  개선된 프롬프트: {improved_score:.1f}/100")
    print(f"  개선 효과: {improved_score - original_score:+.1f}점 ({((improved_score - original_score) / original_score * 100):+.1f}%)")
    
    if improved_score > original_score:
        print("✅ 프롬프트 최적화 성공!")
    else:
        print("⚠️ 추가 최적화가 필요합니다.")

else:
    print("⚠️ API 키가 없어 성능 비교를 건너뜁니다.")

print("\n" + "=" * 50)

## 1️⃣1️⃣ 실무 시나리오: 고객지원 챗봇 프롬프트 최적화

실제 고객지원 시나리오에서 프롬프트 최적화를 적용해봅시다.

In [ ]:
print("🎯 실무 시나리오: 고객지원 챗봇 최적화")
print("=" * 50)

# 고객지원 페르소나 생성
customer_support_persona = PersonaConfig(
    name="customer_support_agent",
    role="전문 고객지원 상담사",
    tone="친절하고 전문적인",
    expertise=["제품 지식", "문제 해결", "고객 소통"],
    constraints=[
        "항상 고객의 입장에서 생각",
        "구체적인 해결책 제시",
        "추가 도움 제안",
        "브랜드 가이드라인 준수"
    ],
    examples=[
        {
            "input": "제품이 작동하지 않아요",
            "output": "불편을 끼쳐드려 죄송합니다. 구체적으로 어떤 문제가 발생했는지 알려주시면 즉시 해결 방법을 안내해드리겠습니다."
        }
    ],
    brand_guidelines={
        "tone": "professional_caring",
        "values": ["고객만족", "신뢰", "혁신", "품질"],
        "forbidden_words": ["불가능", "안됨", "문제", "실패"],
        "preferred_words": ["해결", "지원", "도움", "개선"]
    }
)

# 고객지원 템플릿 변수 설정
cs_variables = {
    "company_name": "TechSolutions",
    "agent_name": "김민수",
    "brand_tone": "친근하면서도 전문적인",
    "brand_values": ["고객만족", "신뢰성", "혁신"],
    "forbidden_words": ["불가능", "안됨", "문제"],
    "inquiry_category": "기술문의",
    "customer_message": "앱이 계속 종료되어서 사용할 수 없어요. 어떻게 해야 하나요?",
    "priority_level": "높음",
    "max_length": 200
}

# 고객지원 프롬프트 생성
try:
    cs_prompt = template_engine.render_prompt("customer_support", cs_variables)
    print("📝 생성된 고객지원 프롬프트:")
    print("-" * 30)
    print(cs_prompt)
    print("-" * 30)
    
    # 프롬프트 최적화 적용
    optimized_cs_prompt = TroubleshootingSolver.add_constraints(
        TroubleshootingSolver.fix_response_format(cs_prompt, "numbered_list"),
        [
            "반드시 해결책을 단계별로 제시",
            "고객의 감정에 공감하며 시작",
            "후속 지원 방법 안내 포함"
        ]
    )
    
    print("\n🔧 최적화된 고객지원 프롬프트:")
    print("-" * 30)
    print(optimized_cs_prompt)
    print("-" * 30)
    
    # AI 응답 생성 및 비교
    if client:
        print("\n🤖 AI 응답 비교")
        print("=" * 30)
        
        customer_inquiry = "앱이 계속 종료되어서 사용할 수 없어요. 정말 화가 나네요."
        
        # 원본 응답
        original_response = generate_ai_response(cs_prompt, customer_inquiry)
        if "error" not in original_response:
            print("📋 원본 프롬프트 응답:")
            print(original_response["response"])
            print(f"토큰: {original_response['tokens_used']}, 시간: {original_response['processing_time']:.2f}초")
        
        print()
        
        # 최적화된 응답
        optimized_response = generate_ai_response(optimized_cs_prompt, customer_inquiry)
        if "error" not in optimized_response:
            print("⭐ 최적화된 프롬프트 응답:")
            print(optimized_response["response"])
            print(f"토큰: {optimized_response['tokens_used']}, 시간: {optimized_response['processing_time']:.2f}초")
        
        # 품질 비교
        if optimizer and "error" not in original_response and "error" not in optimized_response:
            original_quality = optimizer.calculate_quality_score(
                customer_inquiry, original_response["response"], cs_prompt
            )
            optimized_quality = optimizer.calculate_quality_score(
                customer_inquiry, optimized_response["response"], optimized_cs_prompt
            )
            
            print(f"\n📊 품질 비교:")
            print(f"  원본 품질 점수: {original_quality:.1f}/100")
            print(f"  최적화 품질 점수: {optimized_quality:.1f}/100")
            print(f"  개선 효과: {optimized_quality - original_quality:+.1f}점")

except Exception as e:
    print(f"❌ 고객지원 프롬프트 생성 실패: {e}")

print("\n" + "=" * 50)

## 1️⃣2️⃣ 종합 평가 및 베스트 프랙티스

2차시에서 학습한 내용을 종합하고 실무 적용 가이드를 정리합니다.

In [ ]:
print("🎉 2차시 종합 평가 및 베스트 프랙티스")
print("=" * 50)

# 학습한 기능들 요약
print("✅ 구현된 주요 기능:")
print("  1. Jinja2 기반 프롬프트 템플릿 엔진")
print(f"     - 템플릿 수: {len(template_engine.templates)}개")
print(f"     - 카테고리: {set(t.category for t in template_engine.templates.values())}")

print("\n  2. 페르소나 관리 시스템")
print(f"     - 페르소나 수: {len(persona_manager.personas)}개")
print(f"     - 브랜드 가이드라인 지원")

if optimizer:
    print("\n  3. 프롬프트 최적화 도구")
    print("     - A/B 테스트 시스템")
    print("     - 품질 점수 측정")
    print("     - 자동 성능 비교")

print("\n  4. 트러블슈팅 도구")
print("     - 프롬프트 압축 (토큰 최적화)")
print("     - 일관성 확보")
print("     - 응답 형식 제어")
print("     - 제약조건 및 예시 추가")

# 베스트 프랙티스
print("\n💡 프롬프트 엔지니어링 베스트 프랙티스:")
print("\n🎯 1. 명확한 역할 정의")
print("   - 구체적인 전문성과 역할 명시")
print("   - 톤앤매너 일관성 유지")
print("   - 브랜드 가이드라인 준수")

print("\n📝 2. 구조화된 템플릿 사용")
print("   - Jinja2 템플릿으로 재사용성 높이기")
print("   - 변수화를 통한 유연성 확보")
print("   - 카테고리별 템플릿 관리")

print("\n🧪 3. 지속적인 테스트 및 개선")
print("   - A/B 테스트로 객관적 비교")
print("   - 품질 메트릭 기반 최적화")
print("   - 사용자 피드백 반영")

print("\n⚠️ 4. 제약조건 및 안전장치")
print("   - 명확한 제약사항 명시")
print("   - 예상치 못한 응답 방지")
print("   - 브랜드 위험 최소화")

print("\n🔧 5. 운영 최적화")
print("   - 토큰 사용량 최적화")
print("   - 응답 속도 개선")
print("   - 일관성 확보 (temperature=0, seed 고정)")

# 실무 적용 가이드
print("\n🚀 실무 적용 가이드:")
print("\n1️⃣ 프로젝트 시작 시")
print("   - 도메인별 페르소나 정의")
print("   - 기본 템플릿 라이브러리 구축")
print("   - 품질 측정 기준 설정")

print("\n2️⃣ 개발 단계")
print("   - 템플릿 기반 프롬프트 생성")
print("   - 다양한 변형 테스트")
print("   - 사용자 시나리오 검증")

print("\n3️⃣ 운영 단계")
print("   - 정기적 A/B 테스트 실시")
print("   - 성능 모니터링")
print("   - 지속적 개선")

# 다음 차시 미리보기
print("\n🔮 3차시 미리보기: RAG 시스템 구현")
print("   - 문서 업로드 및 파싱")
print("   - 벡터 임베딩 및 검색")
print("   - 지식 기반 답변 생성")
print("   - 출처 표시 및 인용")

print("\n🎊 2차시 완료! 프롬프트 최적화 마스터가 되셨습니다!")
print("=" * 50)

## 📝 숙제 및 확장 과제

### 🏠 숙제
1. **커스텀 템플릿 생성**: 본인의 도메인에 특화된 프롬프트 템플릿 3개 이상 생성
2. **페르소나 A/B 테스트**: 서로 다른 페르소나로 같은 태스크 수행 후 성능 비교
3. **브랜드 가이드라인 적용**: 실제 기업의 브랜드 가이드라인을 반영한 프롬프트 개발

### 🚀 확장 과제
1. **다국어 템플릿 시스템**: 여러 언어를 지원하는 템플릿 엔진 확장
2. **동적 프롬프트 생성**: 사용자 컨텍스트에 따른 실시간 프롬프트 조정
3. **프롬프트 버전 관리**: Git과 연동된 프롬프트 버전 관리 시스템
4. **자동 최적화**: 머신러닝 기반 프롬프트 자동 최적화

### 📚 추천 자료
- [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)
- [Jinja2 Template Documentation](https://jinja.palletsprojects.com/)
- [A/B Testing Best Practices](https://blog.hubspot.com/marketing/how-to-do-a-b-testing)

---

**다음 시간에는 RAG(Retrieval-Augmented Generation) 시스템을 구현하여 문서 기반 질의응답 챗봇을 만들어봅시다! 🔍📚**